In [1]:
import nltk
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import random
import time

class MentalHealthChatbot:
    def __init__(self):
        """Initialize the chatbot with necessary components"""
        self.setup_nltk()
        self.sia = SentimentIntensityAnalyzer()
        self.setup_patterns_and_responses()

    def setup_nltk(self):
        """Download necessary NLTK data"""
        try:
            nltk.data.find('vader_lexicon')
        except LookupError:
            print("Downloading required NLTK data...")
            nltk.download('vader_lexicon', quiet=True)
            nltk.download('punkt', quiet=True)

    def setup_patterns_and_responses(self):
        """Define patterns and corresponding responses for different emotional states"""


        self.crisis_patterns = [
            r'\b(suicide|kill myself|end it all|hurt myself|self harm|want to die)\b',
            r'\b(no point|give up|can\'t go on|end my life)\b'
        ]


        self.distress_patterns = {
            'anxiety': [
                r'\b(anxious|anxiety|panic|worried|stress|nervous|overwhelmed)\b',
                r'\b(can\'t breathe|heart racing|panic attack)\b'
            ],
            'depression': [
                r'\b(depressed|sad|hopeless|empty|lonely|worthless)\b',
                r'\b(can\'t sleep|no energy|don\'t care|numb)\b'
            ],
            'anger': [
                r'\b(angry|furious|rage|mad|frustrated|irritated)\b',
                r'\b(want to scream|so mad|can\'t stand)\b'
            ],
            'physical': [
                r'\b(sick|ill|unwell|tired|exhausted|headache)\b',
                r'\b(can\'t eat|stomach|pain|hurt)\b'
            ]
        }


        self.responses = {
            'crisis': {
                'message': [
                    "I'm really concerned about you. Your feelings are valid, but please reach out for immediate help.",
                    "You matter, and there are people who want to help you through this difficult time.",
                    "These feelings can be overwhelming, but you don't have to face them alone."
                ],
                'suggestions': [
                    "Please contact a crisis helpline immediately",
                    "Reach out to a trusted friend, family member, or counselor",
                    "Go to your nearest emergency room if you're in immediate danger",
                    "Call emergency services if you need immediate assistance"
                ],
                'resources': [
                    "National Suicide Prevention Lifeline: https://suicidepreventionlifeline.org/ (988)",
                    "Crisis Text Line: https://www.crisistextline.org/ (Text HOME to 741741)",
                    "International Association for Suicide Prevention: https://www.iasp.info/resources/Crisis_Centres/"
                ],
                'books': [
                    "\"Reasons to Stay Alive\" by Matt Haig - A powerful memoir about overcoming depression and suicidal thoughts",
                    "\"The Noonday Demon\" by Andrew Solomon - An in-depth exploration of depression and recovery",
                    "\"When Darkness Falls\" by Grandin & Johnson - A guide to surviving and thriving after life's darkest moments",
                    "\"Darkness Visible\" by William Styron - A profound memoir about depression and recovery"
                ]
            },
            'anxiety': {
                'message': [
                    "I hear that you're feeling anxious. Anxiety can be really challenging, but there are ways to manage it.",
                    "It sounds like you're going through a stressful time. Your feelings are completely valid.",
                    "Anxiety can feel overwhelming, but remember that these feelings will pass."
                ],
                'suggestions': [
                    "Try deep breathing: inhale for 4 counts, hold for 4, exhale for 6",
                    "Ground yourself using the 5-4-3-2-1 technique (5 things you see, 4 you hear, etc.)",
                    "Take a short walk or do gentle stretching",
                    "Practice mindfulness or meditation for a few minutes",
                    "Write down your thoughts to help organize them"
                ],
                'resources': [
                    "Anxiety and Depression Association: https://adaa.org/",
                    "Headspace (meditation app): https://www.headspace.com/",
                    "Calm (relaxation app): https://www.calm.com/",
                    "Mental Health America: https://www.mhanational.org/conditions/anxiety-disorders"
                ],
                'books': [
                    "\"The Anxiety and Worry Workbook\" by David A. Clark - Practical cognitive strategies for managing anxiety",
                    "\"Dare: The New Way to End Anxiety\" by Barry McDonagh - A fresh approach to overcoming anxiety and panic",
                    "\"The Mindful Way Through Anxiety\" by Susan Orsillo - Mindfulness techniques for anxiety management",
                    "\"Feel the Fear and Do It Anyway\" by Susan Jeffers - Building confidence and overcoming fear-based thinking",
                    "\"The Worry Cure\" by Robert Leahy - Seven steps to stop worry from stopping you",
                    "\"Anxiety as an Ally\" by Dan Ryckert - A humorous and practical guide to managing anxiety"
                ]
            },
            'depression': {
                'message': [
                    "I'm sorry you're feeling this way. Depression can make everything feel harder, but you're not alone.",
                    "Your feelings are valid, and it's okay to not be okay sometimes.",
                    "Depression can feel isolating, but reaching out like this shows strength."
                ],
                'suggestions': [
                    "Try to maintain a small daily routine, even if it's just one thing",
                    "Get some sunlight or fresh air, even for a few minutes",
                    "Reach out to someone you trust - a friend, family member, or counselor",
                    "Engage in a small activity you used to enjoy",
                    "Practice self-compassion - treat yourself with kindness"
                ],
                'resources': [
                    "National Alliance on Mental Illness: https://www.nami.org/",
                    "Depression and Bipolar Support Alliance: https://www.dbsalliance.org/",
                    "Mental Health America: https://www.mhanational.org/conditions/depression",
                    "Psychology Today (find a therapist): https://www.psychologytoday.com/"
                ],
                'books': [
                    "\"Feeling Good\" by David D. Burns - The classic guide to cognitive behavioral therapy for depression",
                    "\"The Depression Cure\" by Stephen Ilardi - Six steps to beating depression without drugs",
                    "\"Mind Over Mood\" by Dennis Greenberger - Practical workbook for managing depression and anxiety",
                    "\"Lost Connections\" by Johann Hari - Understanding the real causes of depression and anxiety",
                    "\"The Upward Spiral\" by Alex Korb - Using neuroscience to reverse depression",
                    "\"Maybe You Should Talk to Someone\" by Lori Gottlieb - A therapist's journey through her own therapy"
                ]
            },
            'anger': {
                'message': [
                    "It sounds like you're dealing with some intense anger. These feelings are valid.",
                    "Anger can be a difficult emotion to manage, but there are healthy ways to process it.",
                    "I understand you're frustrated. Let's think about some ways to channel these feelings."
                ],
                'suggestions': [
                    "Take slow, deep breaths to help calm your nervous system",
                    "Try physical exercise like going for a walk or doing jumping jacks",
                    "Write about your feelings in a journal",
                    "Talk to someone you trust about what's bothering you",
                    "Practice progressive muscle relaxation"
                ],
                'resources': [
                    "American Psychological Association - Anger Management: https://www.apa.org/topics/anger/control",
                    "Mental Health America - Anger: https://www.mhanational.org/anger",
                    "Mindfulness-Based Stress Reduction: https://www.mindfulnessmbs.com/"
                ],
                'books': [
                    "\"Anger Management Workbook\" by Les Carter - Practical exercises for controlling anger",
                    "\"The Cow in the Parking Lot\" by Leonard Scheff - A zen approach to overcoming anger",
                    "\"Rage: A Step-by-Step Guide\" by Ronald Potter-Efron - Understanding and managing intense anger",
                    "\"The Dance of Anger\" by Harriet Lerner - A woman's guide to changing patterns of intimate relationships",
                    "\"Angry All the Time\" by Ronald Potter-Efron - An emergency guide to anger control",
                    "\"The Gentle Art of Verbal Self-Defense\" by Suzette Haden Elgin - Dealing with verbal aggression"
                ]
            },
            'physical': {
                'message': [
                    "I'm sorry you're not feeling well. Physical symptoms can sometimes be connected to emotional stress.",
                    "It sounds like you're dealing with physical discomfort. Taking care of your body is important.",
                    "Physical and mental health are connected. Let's think about some gentle ways to help."
                ],
                'suggestions': [
                    "Make sure you're staying hydrated and eating regularly",
                    "Try to get adequate rest and sleep",
                    "Consider seeing a healthcare provider if symptoms persist",
                    "Practice gentle relaxation techniques",
                    "Take breaks and don't push yourself too hard"
                ],
                'resources': [
                    "WebMD Symptom Checker: https://symptoms.webmd.com/",
                    "Mayo Clinic Health Information: https://www.mayoclinic.org/",
                    "National Health Service (NHS): https://www.nhs.uk/",
                    "Teladoc (online medical consultations): https://www.teladoc.com/"
                ],
                'books': [
                    "\"The Body Keeps the Score\" by Bessel van der Kolk - How trauma affects the body and healing",
                    "\"Full Catastrophe Living\" by Jon Kabat-Zinn - Using mindfulness to face stress, pain, and illness",
                    "\"When the Body Says No\" by Gabor Maté - The connection between stress and disease",
                    "\"The Mindbody Prescription\" by John Sarno - Understanding psychosomatic symptoms",
                    "\"Heal Your Body\" by Louise Hay - Mental causes for physical illness and healing affirmations",
                    "\"The Healing Path\" by Marc Ian Barasch - Using illness as a catalyst for personal growth"
                ]
            }
        }


        self.wellness_books = [
            "\"The Happiness Project\" by Gretchen Rubin - A year-long journey to discover happiness",
            "\"Atomic Habits\" by James Clear - Building good habits and breaking bad ones",
            "\"The Power of Now\" by Eckhart Tolle - Living in the present moment",
            "\"Man's Search for Meaning\" by Viktor Frankl - Finding purpose in life's challenges",
            "\"The 7 Habits of Highly Effective People\" by Stephen Covey - Personal development principles",
            "\"Mindset\" by Carol Dweck - The psychology of success and growth mindset"
        ]


        self.positive_responses = [
            "That's wonderful to hear! It sounds like you're doing well.",
            "I'm glad you're feeling positive! Keep up the good energy.",
            "It's great that you're in a good headspace. How can I help you today?",
            "That sounds really positive! Is there anything specific you'd like to talk about?"
        ]


        self.default_responses = [
            "I'm here to listen. Can you tell me more about how you're feeling?",
            "I want to help. Could you share what's on your mind?",
            "I'm here for you. What's been going on lately?",
            "How are you feeling today? I'm here to support you."
        ]

    def detect_crisis(self, text):
        """Check if the input indicates a crisis situation"""
        text_lower = text.lower()
        for pattern in self.crisis_patterns:
            if re.search(pattern, text_lower):
                return True
        return False

    def detect_distress_type(self, text):
        """Identify the type of distress from user input"""
        text_lower = text.lower()

        for emotion, patterns in self.distress_patterns.items():
            for pattern in patterns:
                if re.search(pattern, text_lower):
                    return emotion
        return None

    def analyze_sentiment(self, text):
        """Analyze sentiment using VADER"""
        scores = self.sia.polarity_scores(text)
        return scores

    def generate_response(self, user_input):
        """Generate appropriate response based on user input"""
        start_time = time.time()


        if self.detect_crisis(user_input):
            response_type = 'crisis'
        else:

            distress_type = self.detect_distress_type(user_input)
            if distress_type:
                response_type = distress_type
            else:

                sentiment = self.analyze_sentiment(user_input)
                if sentiment['compound'] <= -0.5:
                    response_type = 'depression'
                elif sentiment['compound'] <= -0.1:
                    response_type = 'anxiety'
                elif sentiment['compound'] >= 0.1:
                    response = f"{random.choice(self.positive_responses)}\n\n"
                    response += "Here's a book recommendation for continued growth:\n"
                    response += f"📚 {random.choice(self.wellness_books)}"
                    return response, time.time() - start_time
                else:
                    return random.choice(self.default_responses), time.time() - start_time


        response_data = self.responses[response_type]

        response = f"{random.choice(response_data['message'])}\n\n"
        response += "Here are some suggestions that might help:\n"


        suggestions = random.sample(response_data['suggestions'],
                                  min(3, len(response_data['suggestions'])))
        for i, suggestion in enumerate(suggestions, 1):
            response += f"{i}. {suggestion}\n"

        response += "\nHelpful resources:\n"


        resources = random.sample(response_data['resources'],
                                min(2, len(response_data['resources'])))
        for resource in resources:
            response += f"• {resource}\n"


        response += "\nRecommended reading:\n"
        books = random.sample(response_data['books'],
                            min(2, len(response_data['books'])))
        for book in books:
            response += f"📚 {book}\n"

        if response_type == 'crisis':
            response += "\n⚠ If you're in immediate danger, please call emergency services or go to your nearest emergency room."

        response_time = time.time() - start_time
        return response, response_time

    def run(self):
        """Main chatbot loop"""
        print("=" * 60)
        print("🤗 MENTAL HEALTH SUPPORT CHATBOT")
        print("=" * 60)
        print("Hello! I'm here to provide support and resources for your mental wellness.")
        print("You can talk to me about how you're feeling, and I'll try to help.")
        print("Type 'quit', 'bye', or 'exit' to end our conversation.\n")

        conversation_count = 0

        while True:
            try:
                user_input = input("You: ").strip()


                if user_input.lower() in ['quit', 'bye', 'exit', 'goodbye']:
                    print("\nBot: Take care of yourself! Remember, it's okay to reach out for help.")
                    print("If you need immediate assistance, don't hesitate to contact a mental health professional.")
                    print("You matter, and you're not alone. 💙")
                    break

                if not user_input:
                    print("Bot: I'm here when you're ready to talk.")
                    continue


                response, response_time = self.generate_response(user_input)
                conversation_count += 1

                print(f"\nBot: {response}")


                if response_time < 1.0:
                    print(f"[Response time: {response_time:.3f}s ✓]")
                else:
                    print(f"[Response time: {response_time:.3f}s]")

                print("-" * 40)

            except KeyboardInterrupt:
                print("\n\nBot: Take care! Remember that help is always available.")
                break
            except Exception as e:
                print(f"Bot: I'm sorry, I encountered an error. Please try again.")
                print(f"Error: {e}")

def main():
    """Initialize and run the chatbot"""
    try:
        chatbot = MentalHealthChatbot()
        chatbot.run()
    except Exception as e:
        print(f"Failed to initialize chatbot: {e}")
        print("Make sure yohi u have NLTK installed: pip install nltk")

if __name__ == "__main__":
    main()

🤗 MENTAL HEALTH SUPPORT CHATBOT
Hello! I'm here to provide support and resources for your mental wellness.
You can talk to me about how you're feeling, and I'll try to help.
Type 'quit', 'bye', or 'exit' to end our conversation.

You: hi

Bot: How are you feeling today? I'm here to support you.
[Response time: 0.002s ✓]
----------------------------------------
You: sick

Bot: I'm sorry you're not feeling well. Physical symptoms can sometimes be connected to emotional stress.

Here are some suggestions that might help:
1. Consider seeing a healthcare provider if symptoms persist
2. Try to get adequate rest and sleep
3. Practice gentle relaxation techniques

Helpful resources:
• WebMD Symptom Checker: https://symptoms.webmd.com/
• Teladoc (online medical consultations): https://www.teladoc.com/

Recommended reading:
📚 "The Body Keeps the Score" by Bessel van der Kolk - How trauma affects the body and healing
📚 "The Healing Path" by Marc Ian Barasch - Using illness as a catalyst for perso